In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

sys.path.append(os.path.abspath('../src'))
import utils as util

# Definindo tema e cores
px.defaults.template = "plotly_white"
px.defaults.color_discrete_sequence = [util.barra_azul, util.barra_vermelha] 

df = pd.read_csv('../data/databaseElisa.csv', sep=';', decimal=',', thousands='.', 
                 index_col=None)

df.tail()


,data,fazenda,almoco,janta,cafe,lanche,vlrAlmoco,vlrJanta,vlrCafe,vlrLanche,total
3065,11/05/2025,Silo,5,6,5,0,20.0,20.0,8.5,8.5,262.5
3066,12/05/2025,Santa Elisa,5,1,6,1,20.0,20.0,8.5,8.5,179.5
3067,12/05/2025,Silo,12,6,14,0,20.0,20.0,8.5,8.5,479.0
3068,13/05/2025,Santa Elisa,9,2,6,1,20.0,20.0,8.5,8.5,279.5
3069,13/05/2025,Silo,12,6,14,0,20.0,20.0,8.5,8.5,479.0


In [2]:
util.generate_metadata(df)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,almoco,int64,0,0.0,343
1,janta,int64,0,0.0,236
2,cafe,int64,0,0.0,317
3,lanche,int64,0,0.0,68
4,vlrAlmoco,float64,0,0.0,5
5,vlrJanta,float64,0,0.0,5
6,vlrCafe,float64,0,0.0,5
7,vlrLanche,float64,0,0.0,5
8,total,float64,0,0.0,2718
9,data,object,0,0.0,1535


In [3]:
# Descrição estatística das colunas numéricasdf
cols_to_describe = ['almoco', 'janta', 'cafe', 'lanche', 'vlrAlmoco', 'vlrJanta', 'vlrCafe', 'vlrLanche', 'total']

df[cols_to_describe].describe()

,almoco,janta,cafe,lanche,vlrAlmoco,vlrJanta,vlrCafe,vlrLanche,total
count,3070.000000,3070.000000,3070.000000,3070.000000,3070.000000,3070.000000,3070.000000,3070.000000,3070.000000
mean,92.784691,53.519218,90.430619,13.315309,16.422997,16.422997,7.406254,7.406254,2975.872769
std,76.046257,51.519234,67.105843,15.135026,2.733533,2.733533,0.878378,0.878378,2212.446115
min,0.000000,0.000000,0.000000,0.000000,13.500000,13.500000,6.000000,6.000000,0.000000
25%,36.000000,17.250000,41.000000,1.000000,14.200000,14.200000,6.800000,6.800000,1272.025000
50%,72.000000,34.000000,73.000000,7.000000,14.200000,14.200000,6.800000,6.800000,2423.850000
75%,123.000000,72.000000,117.000000,22.000000,19.000000,19.000000,8.500000,8.500000,3994.625000
max,403.000000,257.000000,355.000000,88.000000,20.000000,20.000000,8.500000,8.500000,11236.000000


In [4]:
util.lista_valores_unicos(df, 'fazenda')

,Coluna,Categoria,Cardinalidade,Porcentagem
0,fazenda,Santa Elisa,1535,50.00%
1,fazenda,Silo,1535,50.00%


In [5]:
df.columns = df.columns.str.strip().str.lower()

In [6]:
# 002. Quantidade total de refeições por fazenda
qtd_cols = ['almoco', 'janta', 'lanche', 'cafe']
df_qtd_fazenda = df.groupby('fazenda')[qtd_cols].sum().reset_index()
df_qtd_fazenda['total_refeicoes'] = df_qtd_fazenda[qtd_cols].sum(axis=1)

fig1 = px.bar(df_qtd_fazenda, x='fazenda', y='total_refeicoes', color='fazenda',
              title='Total de Refeições por Fazenda', text_auto=True)
fig1.show()

In [7]:
# 003. Evolução temporal do total de vendas
df_data_total = df.groupby('data')['total'].sum().reset_index()
fig2 = px.line(df_data_total, x='data', y='total',
               title='Total de Vendas ao Longo do Tempo')
fig2.show()

In [8]:
# 004. Comparativo de ticket médio (valor unitário) entre fazendas
df['vlr_unit_almoco'] = df['vlralmoco'] / df['almoco'].replace(0, pd.NA)
df['vlr_unit_janta'] = df['vlrjanta'] / df['janta'].replace(0, pd.NA)
df['vlr_unit_lanche'] = df['vlrlanche'] / df['lanche'].replace(0, pd.NA)
df['vlr_unit_cafe'] = df['vlrcafe'] / df['cafe'].replace(0, pd.NA)

df_melt = df.melt(id_vars=['fazenda'], value_vars=[
    'vlr_unit_almoco', 'vlr_unit_janta', 'vlr_unit_lanche', 'vlr_unit_cafe'],
    var_name='refeicao', value_name='vlr_unitario')

fig3 = px.box(df_melt, x='refeicao', y='vlr_unitario', color='fazenda',
              title='Distribuição do Valor Unitário por Tipo de Refeição e Fazenda')
fig3.show()

In [9]:
# 005. Análise sazonal: média de vendas por mês
df['data'] = pd.to_datetime(df['data'], dayfirst=True, errors='coerce')
df['mes'] = df['data'].dt.to_period('M').astype(str)
df_mes = df.groupby('mes')['total'].mean().reset_index()

fig4 = px.line(df_mes, x='mes', y='total', markers=True,
               title='Média de Vendas por Mês')
fig4.update_xaxes(type='category')
fig4.show()

In [10]:
# 006. Análise percentual de participação das fazendas no total vendido
df_total_faz = df.groupby('fazenda')['total'].sum().reset_index()
df_total_faz['percentual'] = df_total_faz['total'] / df_total_faz['total'].sum() * 100

fig5 = px.pie(df_total_faz, names='fazenda', values='percentual', color='fazenda',
              title='Participação de Cada Fazenda no Total Vendido (%)')
fig5.show()

In [11]:
# 007. Correlação entre quantidade e valor total das refeições
fig6 = px.scatter(df, x='almoco', y='total', color='fazenda',
                  title='Quantidade x Valor Total (Almoço)')
fig6.show()

fig7 = px.scatter(df, x='janta', y='total', color='fazenda',
                  title='Quantidade x Valor Total (Janta)')
fig7.show()

fig8 = px.scatter(df, x='cafe', y='total', color='fazenda',
                  title='Quantidade x Valor Total (Cafe)')
fig8.show()

fig9 = px.scatter(df, x='lanche', y='total', color='fazenda',
                  title='Quantidade x Valor Total (Lanche)')
fig9.show()

In [12]:
# 008. Evolução de cada tipo de refeição ao longo do tempo
df_refeicoes_tempo = df.groupby('data')[qtd_cols].sum().reset_index()
fig8 = px.line(df_refeicoes_tempo, x='data', y=qtd_cols,
               title='Evolução Diária das Refeições por Tipo')
fig8.show()

In [13]:
# 009. Receita média por tipo de refeição
df_receita_media = pd.DataFrame({
    'Tipo': ['Almoço', 'Janta', 'Lanche', 'Café'],
    'Receita Média': [
        df['vlralmoco'].mean(),
        df['vlrjanta'].mean(),
        df['vlrlanche'].mean(),
        df['vlrcafe'].mean()
    ]
})
fig9 = px.bar(df_receita_media, x='Tipo', y='Receita Média',
              title='Receita Média por Tipo de Refeição', text_auto='.2s')
fig9.show()

In [14]:
# 010. Análise de outliers no valor total por fazenda
fig10 = px.box(df, x='fazenda', y='total',
               title='Distribuição do Valor Total por Fazenda')
fig10.show()

In [15]:
# 011. Proporção de cada tipo de refeição no total servido
df_total_refeicoes = df[qtd_cols].sum().reset_index()
df_total_refeicoes.columns = ['Tipo', 'Total Servido']
df_total_refeicoes['Tipo'] = df_total_refeicoes['Tipo'].str.replace('qtd', '', regex=False).str.capitalize()

fig11 = px.pie(df_total_refeicoes, names='Tipo', values='Total Servido', color='Tipo',
               title='Proporção de Cada Tipo de Refeição Servida')
fig11.show()

In [16]:
# 012. Média diária de refeições por fazenda
df_media_diaria = df.groupby('fazenda')[qtd_cols].mean().reset_index()
df_media_diaria = df_media_diaria.melt(id_vars='fazenda', var_name='Tipo', value_name='Média Diária')
df_media_diaria['Tipo'] = df_media_diaria['Tipo'].str.replace('qtd', '', regex=False).str.capitalize()

fig12 = px.bar(df_media_diaria, x='fazenda', y='Média Diária', color='Tipo',
               barmode='group', title='Média Diária de Refeições por Fazenda', text_auto=True)
fig12.show()

In [17]:
#013. Evolução do ticket médio diário por fazenda
df['ticket_medio'] = df['total'] / (df[qtd_cols].sum(axis=1).replace(0, pd.NA))
df_ticket_diario = df.groupby(['data', 'fazenda'])['ticket_medio'].mean().reset_index()

fig13 = px.line(df_ticket_diario, x='data', y='ticket_medio', color='fazenda',
                title='Evolução do Ticket Médio Diário por Fazenda')
fig13.show()


In [18]:
#014. Top 5 datas com maior venda total
df_top_vendas = df.groupby('data')['total'].sum().nlargest(5).reset_index()

fig14 = px.bar(df_top_vendas, x='data', y='total',
               title='Top 5 Datas com Maior Venda Total', text_auto=True)
fig14.show()


In [22]:
#015. Dias da semana com maior venda média
df['dia_semana'] = df['data'].dt.day_name()
df_venda_dia = df.groupby('dia_semana')['total'].mean().reset_index()

df_venda_dia['dia_semana'] = df_venda_dia['dia_semana'].map(util.traducao_dias)


ordem_dias_pt = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo']
df_venda_dia['dia_semana'] = pd.Categorical(df_venda_dia['dia_semana'], categories=ordem_dias_pt, ordered=True)
df_venda_dia = df_venda_dia.sort_values('dia_semana')

fig15 = px.bar(df_venda_dia, x='dia_semana', y='total',
    title='Média de Vendas por Dia da Semana', text_auto=True)

fig15.show()

In [ ]:

# 016. Média diária de quantidade refeições por dia da semana
qtd_cols = ['almoco', 'janta', 'lanche', 'cafe']
df['dia_semana'] = df['data'].dt.day_name()
df_media_semana = df.groupby('dia_semana')[qtd_cols].mean().reset_index()

df_media_semana['dia_semana'] = df_media_semana['dia_semana'].map(util.traducao_dias)

ordem_dias_pt = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo']
df_media_semana['dia_semana'] = pd.Categorical(df_media_semana['dia_semana'], categories=ordem_dias_pt, ordered=True)
df_media_semana = df_media_semana.sort_values('dia_semana')


df_melted = df_media_semana.melt(id_vars='dia_semana', var_name='Tipo', value_name='Média Diária')


df_melted['Tipo'] = df_melted['Tipo'].str.capitalize()


fig16 = px.bar(df_melted, x='dia_semana', y='Média Diária', color='Tipo', barmode='group',
    title='Média Diária de Refeições por Dia da Semana', text_auto=True)

fig16.show()